Yelp scraper using Selenium

In [1]:
#load necessary libraries
from selenium import webdriver
import time
import chromedriver_binary
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re

In [2]:
#disable build check to if latest version of Chrome is not supported
service = Service(service_args=['--disable-build-check'])

#open the window size to simulate mobile view
options = webdriver.ChromeOptions()
options.add_argument("window-size=600,800")
driver = webdriver.Chrome(service=service, options=options)


The chromedriver version (128.0.6582.0) detected in PATH at C:\Users\22507\anaconda3\lib\site-packages\chromedriver_binary\chromedriver.exe might not be compatible with the detected chrome version (126.0.6478.127); currently, chromedriver 126.0.6478.126 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


Click next page button to get more restaurant urls

In [3]:
def get_all_restaurants(url,num_pages):

    driver.get(url)

    restaurant_urls = []

    try:
        for i in range(num_pages):
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.y-css-12ly5yx'))
            )

            restaurants = driver.find_elements(By.CSS_SELECTOR, 'a.y-css-12ly5yx')
            urls = [restaurant.get_attribute('href') for restaurant in restaurants]
            restaurant_urls.extend(urls)
            restaurant_urls = list(set(restaurant_urls))

            # find and click next page button
            try:
                load_next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.pagination-button__09f24__kbFYf.y-css-1ewzev[data-button="true"]'))
                )

                load_next_page_button.click()


                print(f"Page {i + 1}...")
                print("Clicked 'Next Page' button...Loading...")

                WebDriverWait(driver, 10).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, 'button.pagination-button__09f24__kbFYf.y-css-1ewzev[data-button="false"]'))
                )
                time.sleep(3)

            except Exception as e:
                print(f"Next page button not found or an error occurred: {e}")
                break

    except Exception as e:
        print(f"An error occurred: {e}")

    return restaurant_urls


Get detailed information

In [4]:
def get_restaurant_details(url):

    driver.get(url)

    #Restaurant Name
    restaurant_name = ""

    try:
        restaurant_name = driver.find_element(By.CSS_SELECTOR, "h1")
        restaurant_name = restaurant_name.text
    except:
        pass

    #Overall Rating
    overall_rating = ""
    try:
        overall_rating = driver.find_element(By.CSS_SELECTOR, "span.y-css-kw85nd")
        overall_rating = overall_rating.text
    except:
        pass

    #Review Number
    review_number = ""
    try:
        reviews = driver.find_element(By.CSS_SELECTOR, "span.y-css-loq5qn")
        reviews = reviews.text
        match = re.search(r'\d+', reviews)
        review_number = match.group()
    except:
        pass

    #Food Type
    food_type = ""
    try:
        food_type = driver.find_element(By.CSS_SELECTOR, "span.y-css-1w2z0ld")
        food_type = food_type.text
    except:
        pass

    return (url, restaurant_name, overall_rating, review_number, food_type)


For example

In [5]:
# holy_crab_detail = get_restaurant_details("https://www.yelp.com/biz/holycrab-singapore?osq=Restaurants")
# holy_crab_detail

Combine restaurant urls and restaurant details into a list

In [6]:
#from page 1 to ...(max 24)
restaurant_urls = get_all_restaurants("https://www.yelp.com/search?find_desc=Restaurants&find_loc=Singapore",24)

Page 1...
Clicked 'Next Page' button...Loading...
Page 2...
Clicked 'Next Page' button...Loading...
Page 3...
Clicked 'Next Page' button...Loading...
Page 4...
Clicked 'Next Page' button...Loading...
Page 5...
Clicked 'Next Page' button...Loading...
Page 6...
Clicked 'Next Page' button...Loading...
Page 7...
Clicked 'Next Page' button...Loading...
Page 8...
Clicked 'Next Page' button...Loading...
Page 9...
Clicked 'Next Page' button...Loading...
Page 10...
Clicked 'Next Page' button...Loading...
Page 11...
Clicked 'Next Page' button...Loading...
Page 12...
Clicked 'Next Page' button...Loading...
Page 13...
Clicked 'Next Page' button...Loading...
Page 14...
Clicked 'Next Page' button...Loading...
Page 15...
Clicked 'Next Page' button...Loading...
Page 16...
Clicked 'Next Page' button...Loading...
Page 17...
Clicked 'Next Page' button...Loading...
Page 18...
Clicked 'Next Page' button...Loading...
Page 19...
Clicked 'Next Page' button...Loading...
Page 20...
Clicked 'Next Page' button...

In [7]:
restaurants = []

for restaurant_url in restaurant_urls:
    restaurant_detail = get_restaurant_details(restaurant_url)
    restaurants.append(restaurant_detail)

    time.sleep(0.5)

In [8]:
restaurants = pd.DataFrame(restaurants, columns=["URL", "Restaurant Name", "Overall Rating", "Number of Reviews", "Food Type"])
restaurants

,URL,Restaurant Name,Overall Rating,Number of Reviews,Food Type
0,https://www.yelp.com/biz/the-royals-steakhouse...,The Royals Steakhouse,4.5,2,"Halal, Steakhouses"
1,https://www.yelp.com/biz/violet-oon-singapore?...,Violet Oon,4.3,13,"British, Hainan, Singaporean"
2,https://www.yelp.com/biz/taste-of-morroco-sing...,Taste Of Morroco,5.0,1,Moroccan
3,https://www.yelp.com/biz/beach-road-kitchen-si...,Beach Road Kitchen,4.6,17,"Breakfast & Brunch, Chinese, Buffets"
4,https://www.yelp.com/biz/treasures-yi-dian-xin...,Treasures Yi Dian Xin,4.7,3,"Congee, Dim Sum, Noodles"
...,...,...,...,...,...
235,https://www.yelp.com/biz/brunches-cafe-singapo...,Brunches Cafe,4.3,29,"Coffee & Tea, Bakeries, Breakfast & Brunch"
236,https://www.yelp.com/biz/ki-sho-singapore?osq=...,Ki-sho,4.8,12,"Japanese, Sushi Bars"
237,https://www.yelp.com/biz/happy-crab-singapore?...,Happy Crab,4.2,12,"Seafood, Singaporean"
238,https://www.yelp.com/biz/jb-ah-meng-singapore?...,JB Ah Meng,4.6,25,"Chinese, Seafood"


In [9]:
csv_file_path = 'C:/Users/22507/Desktop/NUS/project/restaurants.csv'
restaurants.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"CSV file saved at: {csv_file_path}")

CSV file saved at: C:/Users/22507/Desktop/NUS/project/restaurants.csv


In [10]:
driver.quit()